In [1]:
#enveloping potential
# simple Example plot Enveloped Potential with two Harmonic Oscilators
##Imports:
import os, sys as csys, datetime, tempfile
import math, numpy as np, pandas as pd
import scipy.constants as const

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm


csys.path.append(os.getcwd()+"/../../..")

#Ensembler
import ensembler.potentials.TwoD as pot2D

from ensembler.samplers.stochastic import metropolisMonteCarloIntegrator
from ensembler.conditions.box_conditions import periodicBoundaryCondition
from ensembler.system.basic_system import system

import ensembler.visualisation.plotPotentials as exPlot

%matplotlib inline

In [2]:
#ENERGIES Sampling
def write_out_etraj(traj, out_path, V1, V2):
    visited_positions =  np.array(list(map(np.array, traj.position)))
    state1 = V1.ene(visited_positions)
    state2 = V2.ene(visited_positions)
    Vrenergies = np.array(traj.total_potential_energy)

    out_file = open(out_path, "w")
    out_file.write("t\tV1\tV2\tVr\n")#HEADER
    for t, V1, V2, Vr in zip(traj.index, state1, state2, Vrenergies):
        out_file.write(str(t)+"\t"+str(V1)+"\t"+str(V2)+"\t"+str(Vr)+"\n")

    out_file.close()

In [3]:
    
#METROPOLIS CRITERION
##random part of Metropolis Criterion:
_defaultRandomness = lambda V1, V2: (np.random.rand() <= np.exp(-1.0 / (const.gas_constant / 1000.0 * 298) * (V1 - V2))) #pseudocount  for equal energies
##default Metropolis Criterion
_defaultMetropolisCriterion = lambda V1, V2: (V1 < V2 or _defaultRandomness(V1, V2))

    
def MC_exchange(sys1, sys2):
    s1 = sys1.potential.s
    s2 = sys2.potential.s

    V1_s1 = sys1.potential.ene(sys1._currentPosition)

    V2_s2 = sys2.potential.ene(sys2._currentPosition)

    sys1.potential.s = s2
    V1_s2 = sys1.potential.ene(sys1._currentPosition)

    sys2.potential.s = s1
    V2_s1 = sys2.potential.ene(sys2._currentPosition)

    sys1.potential.s = s1
    sys2.potential.s = s2

    exchange = _defaultMetropolisCriterion((V2_s1+V1_s2),(V1_s1+V2_s2))
    return exchange

In [4]:
#Simulation Setup
##Build System
periodic_bound = periodicBoundaryCondition(boundary=[[-180,180], [-180,180]])
integrator = metropolisMonteCarloIntegrator(fixed_step_size=[np.deg2rad(1),np.deg2rad(1)], max_iteration_tillAccept=1000)


##STEPS
replicas = 10
each_sim = 20#0 #steps between trials
trials = 10#00  #exchange attempts

#s_est = 0.609/Delta U_barrier
optimal_ses = {2.5: 0.122,
               5: 0.061,
               10: 0.03,
               20: 0.03, #0.015
               50: 0.006}

initial_position = None



In [10]:
#run multiple replicas
root_tmp_dir = tempfile.gettempdir()+"/edsSim"
if(not os.path.exists(root_tmp_dir)):
    os.mkdir(root_tmp_dir)
os.chdir(root_tmp_dir)

tmp_dir = root_tmp_dir+"/re_eds_simulation"
if(not os.path.exists(tmp_dir)):
    os.mkdir(tmp_dir)
os.chdir(tmp_dir)

print(tmp_dir)


for barrier in tqdm(optimal_ses, desc="Barrier: "):
    approach_dir = tmp_dir+"/simulation_with_"+str(barrier)+"kt_barrier"
    if(not os.path.exists(approach_dir)): os.mkdir(approach_dir)


    start_t = datetime.datetime.now()
    
    #build potential
    amplitude = barrier/2
    shift  = np.rad2deg(0.5*math.pi)
    Eoff = [0,0]

    V1 = pot2D.wavePotential(phase_shift=(shift,shift), multiplicity=(2.0, 2.0), amplitude=(amplitude, amplitude))
    V2 = pot2D.wavePotential(phase_shift=(0.0, 0.0), multiplicity=(2.0, 2.0), amplitude=(amplitude, amplitude))
    edsPot1 = pot2D.envelopedPotential(V_is=[V1, V2], s=1.0, eoff=Eoff)
    edsPot2 = pot2D.envelopedPotential(V_is=[V1, V2], s=1.0, eoff=Eoff)
    
    
    start=0
    syst = None
    for replica in tqdm(range(start, replicas), desc="Replicas: "):
        ##INIT REPLICA
        ###fILE hANDLING
        replica_out = approach_dir+"/replica_"+str(replica)
        if(not os.path.exists(replica_out)):
            os.mkdir(replica_out)

        ###Build System
        sys1=system(potential=edsPot1, sampler=integrator, conditions=[periodic_bound])
        sys2=system(potential=edsPot2, sampler=integrator, conditions=[periodic_bound])

        sys1.potential.s = 0.5
        sys2.potential.s = 0.01

        ## SIMULATE REPLICA
        start_sim = datetime.datetime.now()
        s_val_posDict = {}  #OUT
        current_s_index = 0
        current_s_direction = 1

        for trial in tqdm(range(trials), desc="Current Simulation of replica "+str(replica)+": ",):
            cur_state1 = sys1.simulate(each_sim, withdraw_traj=False, verbosity=False)
            cur_state2 = sys2.simulate(each_sim, withdraw_traj=False, verbosity=False)

            if(MC_exchange(sys1, sys2)):
                sys1._currentPosition = cur_state2.position
                sys2._currentPosition = cur_state1.position
            else:
                pass

        s_val_posDict.update({sys2.potential.s[0] : sys2.trajectory})
        s_val_posDict.update({sys1.potential.s[0] : sys1.trajectory})

        end_sim = datetime.datetime.now()
        duration_sim = end_sim-start_sim

        #plotting:
        print("ploting")
        start_plot = datetime.datetime.now()
        fig = exPlot.plot_2D_2State_EDS_potential_sDependency(s_val_posDict, eds_pot=edsPot1, plot_trajs=True, out_path=replica_out+"/s_dependent_sampling_relBarrier_"+str(barrier)+"kT_withTraj.png")
        end_plot = datetime.datetime.now()
        duration_plot = end_plot-start_plot

        for s in s_val_posDict:
            write_out_etraj(traj=s_val_posDict[s], out_path=replica_out+"/replica_traj_s"+str(s)+".dat", V1=sys1.potential.V_is[0], V2=sys1.potential.V_is[1])
        #del sys1, sys2
        break


    end_t = datetime.datetime.now()
    duration = end_t-start_t
    print("Done - duration: ", duration)
    print("Done - simulation duration: ", duration_sim)
    print("Done - plotting duration: ", duration_plot)

print(root_tmp_dir)

C:\Users\benja\AppData\Local\Temp/edsSim/re_eds_simulation



ploting
[-5.76826261 -5.75080931 -5.76826261 -5.7857159  -5.80316919 -5.82062248
 -5.83807578 -5.82062248 -5.80316919 -5.82062248 -5.80316919 -5.7857159
 -5.80316919 -5.82062248 -5.80316919 -5.82062248 -5.83807578 -5.82062248
 -5.80316919 -5.82062248 -5.83807578 -7.15825155 -7.17570484 -7.15825155
 -7.14079826 -7.12334497 -7.14079826 -7.12334497 -7.10589167 -7.12334497
 -7.10589167 -7.12334497 -7.14079826 -7.12334497 -7.14079826 -7.15825155
 -7.14079826 -7.12334497 -7.10589167 -7.08843838 -7.10589167 -5.75080931
 -5.76826261 -5.75080931 -5.73335602 -5.71590273 -5.73335602 -5.71590273
 -5.69844944 -5.71590273 -5.69844944 -5.68099614 -5.66354285 -5.64608956
 -5.66354285 -5.64608956 -5.62863627 -5.61118297 -5.62863627 -5.61118297
 -5.59372968 -7.29787789 -7.31533118 -7.33278448 -7.35023777 -7.36769106
 -7.35023777 -7.36769106 -7.38514435 -7.40259765 -7.38514435 -7.40259765
 -7.38514435 -7.40259765 -7.42005094 -7.40259765 -7.38514435 -7.40259765
 -7.38514435 -7.36769106 -7.35023777 -5.785


Done - duration:  0:00:17.452141
Done - simulation duration:  0:00:08.802289
Done - plotting duration:  0:00:07.496737



ploting
[-4.20413148 -4.22158478 -4.20413148 -4.18667819 -4.1692249  -4.18667819
 -4.1692249  -4.18667819 -4.20413148 -4.22158478 -4.20413148 -4.22158478
 -4.23903807 -4.25649136 -4.27394465 -4.25649136 -4.27394465 -4.29139795
 -4.30885124 -4.29139795 -4.27394465  9.1273047   9.14475799  9.1273047
  9.14475799  9.1273047   9.10985141  9.1273047   9.10985141  9.09239811
  9.10985141  9.1273047   9.10985141  9.09239811  9.07494482  9.09239811
  9.10985141  9.09239811  9.07494482  9.09239811  9.10985141 -4.3961177
 -4.37866441 -4.36121112 -4.37866441 -4.36121112 -4.34375782 -4.32630453
 -4.34375782 -4.36121112 -4.37866441 -4.3961177  -4.41357099 -4.43102429
 -4.41357099 -4.43102429 -4.41357099 -4.3961177  -4.41357099 -4.43102429
 -4.41357099  9.05749153  9.07494482  9.05749153  9.07494482  9.09239811
  9.07494482  9.05749153  9.04003824  9.05749153  9.04003824  9.02258494
  9.00513165  9.02258494  9.00513165  8.98767836  9.00513165  9.02258494
  9.04003824  9.05749153  9.04003824 -4.3263


Done - duration:  0:00:16.981580
Done - simulation duration:  0:00:08.656452
Done - plotting duration:  0:00:06.967885


...


Done - duration:  0:00:17.777442
Done - simulation duration:  0:00:09.472940
Done - plotting duration:  0:00:07.144465



ploting
[-5.57479298 -5.59224628 -5.57479298 -5.55733969 -5.57479298 -5.59224628
 -5.60969957 -5.62715286 -5.64460615 -5.62715286 -5.64460615 -5.66205945
 -5.67951274 -5.69696603 -5.71441932 -5.73187262 -5.74932591 -5.73187262
 -5.74932591 -5.7667792  -5.74932591 -9.29899134 -9.28153805 -9.29899134
 -9.28153805 -9.26408476 -9.28153805 -9.29899134 -9.31644463 -9.29899134
 -9.31644463 -9.33389793 -9.31644463 -9.33389793 -9.31644463 -9.33389793
 -9.35135122 -9.33389793 -9.31644463 -9.29899134 -9.28153805 -5.80168579
 -5.78423249 -5.7667792  -5.74932591 -5.7667792  -5.74932591 -5.7667792
 -5.74932591 -5.7667792  -5.74932591 -5.73187262 -5.74932591 -5.7667792
 -5.74932591 -5.7667792  -5.74932591 -5.7667792  -5.78423249 -5.7667792
 -5.78423249 -9.19427159 -9.21172488 -9.19427159 -9.21172488 -9.19427159
 -9.21172488 -9.22917817 -9.24663146 -9.22917817 -9.21172488 -9.22917817
 -9.21172488 -9.19427159 -9.21172488 -9.19427159 -9.21172488 -9.19427159
 -9.17681829 -9.159365   -9.17681829 -5.55733


Done - duration:  0:00:07.140945
Done - simulation duration:  0:00:03.574554
Done - plotting duration:  0:00:02.746331



ploting
[-7.3300861  -7.3475394  -7.3300861  -7.3475394  -7.36499269 -7.3475394
 -7.36499269 -7.38244598 -7.39989927 -7.38244598 -7.39989927 -7.38244598
 -7.36499269 -7.3475394  -7.3300861  -7.3475394  -7.36499269 -7.38244598
 -7.36499269 -7.38244598 -7.39989927  0.27042663  0.28787992  0.27042663
  0.25297333  0.23552004  0.25297333  0.23552004  0.21806675  0.23552004
  0.21806675  0.23552004  0.21806675  0.23552004  0.21806675  0.23552004
  0.21806675  0.23552004  0.21806675  0.23552004  0.25297333 -7.31263281
 -7.29517952 -7.27772623 -7.29517952 -7.31263281 -7.3300861  -7.3475394
 -7.36499269 -7.3475394  -7.36499269 -7.38244598 -7.36499269 -7.3475394
 -7.36499269 -7.38244598 -7.39989927 -7.41735257 -7.43480586 -7.45225915
 -7.43480586  0.3402398   0.3227865   0.3402398   0.35769309  0.3402398
  0.3227865   0.30533321  0.3227865   0.30533321  0.28787992  0.30533321
  0.28787992  0.27042663  0.25297333  0.27042663  0.28787992  0.27042663
  0.28787992  0.27042663  0.28787992 -7.487165


Done - duration:  0:00:12.582134
Done - simulation duration:  0:00:07.012366
Done - plotting duration:  0:00:05.096158

C:\Users\benja\AppData\Local\Temp/edsSim


In [6]:
print(root_tmp_dir)


C:\Users\benja\AppData\Local\Temp/edsSim


In [11]:
sys1

In [12]:
sys1.trajectory.head()

,position,temperature,total_system_energy,total_potential_energy,total_kinetic_energy,dhdpos,velocity
0,"[-7.330086104037081, -0.7240010382638431]",298.0,-10.118148,-10.118148,NaN,"[-43.31610027881477, -49.62351272908448]",NaN
1,"[-7.347539396557024, -0.7414543307837864]",298.0,-12.427326,-12.427326,NaN,"[-0.017453292519943295, -0.017453292519943295]",NaN
2,"[-7.330086104037081, -0.7589076233037297]",298.0,-12.549535,-12.549535,NaN,"[0.017453292519943295, -0.017453292519943295]",NaN
3,"[-7.347539396557024, -0.7414543307837864]",298.0,-12.427326,-12.427326,NaN,"[-0.017453292519943295, 0.017453292519943295]",NaN
4,"[-7.364992689076967, -0.7589076233037297]",298.0,-14.029906,-14.029906,NaN,"[-0.017453292519943295, -0.017453292519943295]",NaN
